In [2]:
import requests
import lxml.html as lh
import pandas as pd

### Scrap the table from Wiki

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Get the colunm names first

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print '%d:"%s"'%(i,name)
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [6]:
[len(C) for (title,C) in col]

[0, 0, 0]

### Get the data from HTML table

In [7]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
print("Data Captured")

Data Captured


#### Prepare the datafrmae

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(50)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [9]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [10]:
df.rename(columns={"Neighbourhood\n": "Neighbourhood"}, inplace = True)
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [11]:
df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [12]:
df["Neighbourhood"]= df["Neighbourhood"].str.replace("\n", "")
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
df.shape

(288, 3)

#### Drop all the "Not-assigned" Borough rows

In [14]:
# Drop a row by condition
df_bor = df[df.Borough != 'Not assigned']
df_bor.shape

(211, 3)

#### Sort and group the dataframe by post code

In [15]:
df_sorted = df_bor.sort_values(['Postcode'])
#df_sorted['Neighbourhood']
df_sorted

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
29,M1C,Scarborough,Port Union
28,M1C,Scarborough,Rouge Hill
27,M1C,Scarborough,Highland Creek
42,M1E,Scarborough,Guildwood
43,M1E,Scarborough,Morningside
44,M1E,Scarborough,West Hill
53,M1G,Scarborough,Woburn
62,M1H,Scarborough,Cedarbrae


### If a cell has a 'borough' but a Not assigned 'neighborhood', then the neighborhood will be the same as the 'borough' 

In [16]:
# # If a cell has a 'borough' but a Not assigned 'neighborhood', then the neighborhood will be the same as the 'borough' 
import numpy as np
df_sorted['Neighbourhood'] = np.where(df_sorted['Neighbourhood'] == 'Not assigned', df_sorted['Borough'], df_sorted['Neighbourhood'])
df_sorted

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
29,M1C,Scarborough,Port Union
28,M1C,Scarborough,Rouge Hill
27,M1C,Scarborough,Highland Creek
42,M1E,Scarborough,Guildwood
43,M1E,Scarborough,Morningside
44,M1E,Scarborough,West Hill
53,M1G,Scarborough,Woburn
62,M1H,Scarborough,Cedarbrae


### If More than one neighborhood can exist in one borough, list all neighborhood separated by comma

In [17]:
# If More than one neighborhood can exist in one borough, list all neighborhood separated by comma
grp = df_sorted.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ','.join(x)).reset_index()
df_final = pd.DataFrame(grp)
df_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df_final.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [19]:
# Convert the data type of the column if not appropriate
#df_final['Neighbourhood']= df_final['Neighbourhood'].astype(str)
df_final.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [20]:
df_final

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [21]:
df_final.shape

(103, 3)

### Get the coordinated from CSV into a dataframe

In [22]:
coord_filename = "D:/UserData/z003fe3x/Documents/Geospatial_Coordinates.csv"

In [23]:
df_coord = pd.read_csv(coord_filename, sep = ',')
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [24]:
# Rename the key field (Post code) for the new data frame
df_coord.rename(columns={"Postal Code": "Postcode"}, inplace = True)
df_coord.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

### Merge the 2 dataframe in to 1

In [25]:
df_merge_col = pd.merge(df_final, df_coord, on='Postcode')
df_merge_col

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


### Display Map with Folium

In [29]:
# Install Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.1.23-0, anaconda::openssl-1.1.1b-he774522_1
  - anaconda::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1
  - anaconda::openssl-1.1.1b-he774522_1, defaults::ca-certificates-2019.1.23-0
  - defaults::ca-certificates-2019.1.23-0, defaults::openssl-1.1.1b-he774522_1done

## Package Plan ##

  environment location: C:\Users\z003fe3x\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  conda                                       4.7.11-py37_0 --> 4.7.12-py37_0




conda-4.7.12         | 3

In [31]:
latitude = 43.665860
longitude = -79.383160

venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_merge_col.Latitude, df_merge_col.Longitude, df_merge_col.Postcode):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map